# On the hashing trick

This is a deep-dive on the _hashing trick_, a simple but fascinating technique to scale up your machine learning model.

It was first made popular through the _Vowpal Wabbit_ library ([released in 2007](https://hunch.net/?p=309) by Yahoo researchers).

Useful references:

 - [1] Starting paper: Weinberger, Kilian, et al. "[Feature hashing for large scale multitask learning.](https://icml.cc/Conferences/2009/papers/407.pdf)" Proceedings of the 26th annual international conference on machine learning. 2009.
 - [2] Shi, Qinfeng, et al. "[Hash kernels.](http://proceedings.mlr.press/v5/shi09a/shi09a.pdf)" Artificial intelligence and statistics. PMLR, 2009.
 - [3] Freksen, Casper B., Lior Kamma, and Kasper Green Larsen. "[Fully understanding the hashing trick.](https://proceedings.neurips.cc/paper/2018/file/7e83722522e8aeb7512b7075311316b7-Paper.pdf)" Advances in Neural Information Processing Systems 31 (2018).
 - [4] Criteo's version: Chapelle, Olivier, Eren Manavoglu, and Romer Rosales. "[Simple and scalable response prediction for display advertising.](https://www.cse.fau.edu/~xqzhu/courses/cap6807/reading/response.prediction.pdf)" ACM Transactions on Intelligent Systems and Technology (TIST) 5.4 (2014): 1-34.

 ## Key idea

 Say your data is represented by a high dimensional vector $x$ of possibly unbounded dimension $d$.  
 Hash the features into a lower dimensional feature space $\mathbb{R}^m$ with $\phi : \mathcal{X} \rightarrow \mathbb{R}^m$, with $m \ll d$. 

 ### Weinberger et al's definition

 [1] defines the feature map with two hash functions $h : \mathbb{N} \rightarrow \{1,\dots,m\}$ and $\xi : \mathbb{N} \rightarrow {\pm 1}$ and for a vector $x$, by:
 $$ \phi_i(x) = \sum_{j:h(j)=i} \xi(j)x_j. $$
 
 In other words: for each input feature indexed by $j$, compute a hash on the index to select the output feature index (aka bucket) $h(j)=i$, and another hash to decide on the sign. Accumulate all features in the bucket!

### Chapelle et al's definition

In the context of online advertising, the authors are only using categorical features, such as identifiers (advertiser ids, publisher id, product id...).
The standard _dummy coding_ would yield a vector of $c$ binary values if there are $c$ possible categories for a given feature. That is, the total number of features, or dimensions if using naive encoding, woud be:
$$ d = \sum_{f=1}^F c_f,$$
where $F$ is the number of features, and $c_f$ the number of values that feature $f$ can take.

Then they hash all the features into the same space, using a different function for each feature -- as implemented in Vowpal Wabbit. In practice, they use the same family of hash functions, MurmurHash3, using _f_ as seed. 

Let $x={x_1, \dots, x_F}$, we have :
$$ \phi_i(x) = \sum_{f} [h_f(x) \mod d = i], $$
where $[p]=1$ if predicate $p$ is true, else $[p]=0$.
In other words, the feature map counts the number of categories that fall into that bucket.

## Properties

[1] show that with their definition adding a sign, the hash kernel is unbiased, ie the scalar product in feature space approximates the true scalar product in expectation.

Like [2], they also demonstrate concentration bounds -- ie this hashing trick has limited distortion effect on the features. Previous results showed that the bounds are tighter when the mass of $x$ needs to be well distributed, ie $||x||_\inf / ||x||_2$ needs to be sufficiently small.

[3] gets deeper in the concentration bounds analysis, but notes that "in practice... feature hashing performs well for a larger scope of vectors".

The main advantage is that feature hashing allows for significant storage compression: this is particularly obvious in the case of advertising, where the number of ids grows very quickly, and even more when you take into account cross-features.

## Application to multi-task learning

Let $U$ be a set of different tasks. In multitask learning, we want to learn parameter vectors $w_u$ for each task, as well as a global parameter $w$ such that common patterns are shared across tasks.

[1] shows that the parameters learnt for the tasks $U\backslash\{u\}$, noted $w$, have limited interactions with feature vectors for task $u$, through a concentration bound:
$$
P(|\left< w, \phi_u(x)\right>| > \varepsilon) \leq 2\exp\left(- \frac{\varepsilon^2/2 \cdot m}{sth(||w||\cdot||x||)}\right),
$$
where the good news stems from having this bound decreasing exponentially when $m$ increases, as long as the norms are kept relatively small.

[1] show an example where they learn per-user weights for spam filtering, which can scale to billions of users: instead of having $d \times (|U| + 1)$ weights, they can have only $m \ll d$ weights,by "hashing the weights" into a vector $w_h \in \mathbb{R}^m$:
$$
w_h = \phi_0(w_0) + \sum_{u\in U} \phi_u(w_u)
$$

In [4], multi-task learning is achieved by building conjunction features, implicitly building task-specific vectors. Eg if we want to have a partner-specific model, we use a partner-specific hash function for each feature.

![](./criteo_prediction.png)

In [ ]:
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=10, )
D = [{'dog': 1, 'cat':2, 'elephant':4},{'dog': 2, 'run': 5}]
f = h.transform(D)
f.toarray()